# Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

# Credentials

In [2]:
pg_user_dev = 'postgres'
pg_pass_dev = 'admin'
pg_db_dev = 'postgres'
pg_host_dev = 'localhost'
pg_port_dev = 5432
DEV_DB = f"postgresql://{pg_user_dev}:{pg_pass_dev}@{pg_host_dev}:{pg_port_dev}/{pg_db_dev}"

# Create Engine

In [3]:
# Start the session
engine = create_engine(DEV_DB, echo=False, client_encoding="UTF-8")
Session = sessionmaker(bind=engine)
Session.configure(bind=engine)
session = Session()
meta = MetaData(bind=engine)
Base = declarative_base()

# Delete Schema and Tables

In [4]:
session.execute("DROP SCHEMA IF EXISTS movie_recommender CASCADE;")
session.commit()

In [5]:
session.execute("CREATE SCHEMA IF NOT EXISTS movie_recommender;")
session.commit()

In [6]:
import sys
sys.path.append("../recommender_app")
from main import db

/Users/filipstoja/opt/anaconda3/envs/movie_recommender/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [7]:
db.create_all()

# Declare Tables

In [8]:
# Read in tables from the database
class MovieUsers(Base):
    __table__ = Table('movie_users', meta, autoload=True, schema="movie_recommender")
    ratings = relationship('Ratings', backref='movie_users', lazy=True)
    tags = relationship('Tags', backref='movie_users', lazy=True)

# Read in tables from the database
class Movies(Base):
    __table__ = Table('movies', meta, autoload=True, schema="movie_recommender")
    ratings = relationship('Ratings', backref='movies', lazy=True)
    tags = relationship('Tags', backref='movies', lazy=True)
    links = relationship('Links', backref='movies', uselist=False, lazy=True)


class Ratings(Base):
    __table__ = Table('ratings', meta, autoload=True, schema="movie_recommender")

class Tags(Base):
    __table__ = Table('tags', meta, autoload=True, schema="movie_recommender")

class Links(Base):
    __table__ = Table('links', meta, autoload=True, schema="movie_recommender")

# Read in Data

In [9]:
movies = pd.read_csv("data/movies.csv")
ratings = pd.read_csv('data/ratings.csv')
tags = pd.read_csv('data/tags.csv')
links = pd.read_csv('data/links.csv')

# Upload Movies

In [10]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))

In [12]:
movies_agg = movies.explode('genres')

In [13]:
movies_agg = pd.pivot_table(movies_agg, values = 'movieId', index = 'movieId',
               columns = 'genres', aggfunc = len).reset_index()

In [14]:
movies_agg.columns = ['genre_' + x.lower().replace(" ", "_").replace("-","_").replace("(","").replace(")","") \
                      if x != 'movieId' else x for x in movies_agg.columns]

In [15]:
movies_agg = movies_agg.fillna(0)

In [16]:
for col in [x for x in movies_agg.columns if 'genre_' in x]:
    movies_agg[col] = movies_agg[col].astype(bool)

In [17]:
movies = movies.drop(columns = 'genres').merge(movies_agg, on = 'movieId')

In [18]:
movies.isna().sum()

movieId                   0
title                     0
genre_no_genres_listed    0
genre_action              0
genre_adventure           0
genre_animation           0
genre_children            0
genre_comedy              0
genre_crime               0
genre_documentary         0
genre_drama               0
genre_fantasy             0
genre_film_noir           0
genre_horror              0
genre_imax                0
genre_musical             0
genre_mystery             0
genre_romance             0
genre_sci_fi              0
genre_thriller            0
genre_war                 0
genre_western             0
dtype: int64

In [19]:
movies.nunique()

movieId                   9742
title                     9737
genre_no_genres_listed       2
genre_action                 2
genre_adventure              2
genre_animation              2
genre_children               2
genre_comedy                 2
genre_crime                  2
genre_documentary            2
genre_drama                  2
genre_fantasy                2
genre_film_noir              2
genre_horror                 2
genre_imax                   2
genre_musical                2
genre_mystery                2
genre_romance                2
genre_sci_fi                 2
genre_thriller               2
genre_war                    2
genre_western                2
dtype: int64

In [20]:
# All of the rated movies have a genre
ratings[~ratings['movieId'].isin(movies['movieId'])]

,userId,movieId,rating,timestamp


In [21]:
movies.to_dict(orient='records')[0:10]

[{'movieId': 1,
  'title': 'Toy Story (1995)',
  'genre_no_genres_listed': False,
  'genre_action': False,
  'genre_adventure': True,
  'genre_animation': True,
  'genre_children': True,
  'genre_comedy': True,
  'genre_crime': False,
  'genre_documentary': False,
  'genre_drama': False,
  'genre_fantasy': True,
  'genre_film_noir': False,
  'genre_horror': False,
  'genre_imax': False,
  'genre_musical': False,
  'genre_mystery': False,
  'genre_romance': False,
  'genre_sci_fi': False,
  'genre_thriller': False,
  'genre_war': False,
  'genre_western': False},
 {'movieId': 2,
  'title': 'Jumanji (1995)',
  'genre_no_genres_listed': False,
  'genre_action': False,
  'genre_adventure': True,
  'genre_animation': False,
  'genre_children': True,
  'genre_comedy': False,
  'genre_crime': False,
  'genre_documentary': False,
  'genre_drama': False,
  'genre_fantasy': True,
  'genre_film_noir': False,
  'genre_horror': False,
  'genre_imax': False,
  'genre_musical': False,
  'genre_myster

In [22]:
# Insert all of the stock data into the database
for movie in movies.to_dict(orient="records"):
    movie = Movies(**movie)
    try:
        session.add(movie)
        session.commit()

    except Exception as e:
        session.rollback()
        print(f"Exception inserting {str(movie)} into database: {e}")

# Create and Upload Users

In [23]:
users = ratings.drop_duplicates(subset = ['userId'])
users = users[['userId']]

In [24]:
users

,userId
0,1
232,2
261,3
300,4
516,5
...,...
97364,606
98479,607
98666,608
99497,609


In [25]:
# Insert all of the stock data into the database
for user in users.to_dict(orient="records"):
    user = MovieUsers(**user)
    try:
        session.add(user)
        session.commit()

    except Exception as e:
        session.rollback()
        print(f"Exception inserting {str(movie)} into database: {e}")

# Upload Ratings

In [26]:
ratings['timestamp'] = ratings['timestamp'].apply(datetime.fromtimestamp)

In [27]:
# Insert all of the stock data into the database
for rating in ratings.to_dict(orient="records"):
    rating = Ratings(**rating)
    try:
        session.add(rating)
        session.commit()

    except Exception as e:
        session.rollback()
        print(f"Exception inserting {str(rating)} into database: {e}")

# Upload Tags

In [28]:
tags['timestamp'] = tags['timestamp'].apply(datetime.fromtimestamp)

In [29]:
for tag in tags.to_dict(orient="records"):
    tag = Tags(**tag)
    try:
        session.add(tag)
        session.commit()

    except Exception as e:
        session.rollback()
        print(f"Exception inserting {str(tag)} into database: {e}")

# Upload Links

In [30]:
links = links.replace({np.nan: None})

In [31]:
for link in links.to_dict(orient="records"):
    link = Links(**link)
    try:
        session.add(link)
        session.commit()

    except Exception as e:
        session.rollback()
        print(f"Exception inserting {str(tag)} into database: {e}")

# Reading and Comparing

## Users

In [32]:
users_table = pd.read_sql_table(
    "movie_users",
    con=engine,
    schema = "movie_recommender"
)

In [33]:
users_table = users_table.sort_values(by = 'userId').reset_index(drop = True)
users = users.sort_values(by = 'userId').reset_index(drop = True)

In [34]:
(users_table == users).sum()

userId    610
dtype: int64

## Movies

In [35]:
movies_table = pd.read_sql_table(
    "movies",
    con=engine,
    schema = "movie_recommender"
)

In [36]:
movies_table = movies_table.sort_values(by = 'movieId').reset_index(drop = True)
movies = movies.sort_values(by = 'movieId').reset_index(drop = True)

In [37]:
(movies_table == movies).sum()

movieId                   9742
title                     9742
genre_no_genres_listed    9742
genre_action              9742
genre_adventure           9742
genre_animation           9742
genre_children            9742
genre_comedy              9742
genre_crime               9742
genre_documentary         9742
genre_drama               9742
genre_fantasy             9742
genre_film_noir           9742
genre_horror              9742
genre_imax                9742
genre_musical             9742
genre_mystery             9742
genre_romance             9742
genre_sci_fi              9742
genre_thriller            9742
genre_war                 9742
genre_western             9742
dtype: int64

## Ratings

In [38]:
ratings_table = pd.read_sql_table(
    "ratings",
    con=engine,
    schema = "movie_recommender"
)

In [39]:
ratings_table = ratings_table.drop(columns='id').sort_values(by = ['userId','movieId']).reset_index(drop = True)
ratings = ratings.sort_values(by = ['userId','movieId']).reset_index(drop = True)

In [40]:
(ratings_table == ratings).sum()

userId       100836
movieId      100836
rating       100836
timestamp    100836
dtype: int64

## Tags

In [41]:
tags_table = pd.read_sql_table(
    "tags",
    con=engine,
    schema = "movie_recommender"
)

In [42]:
tags_table = tags_table.drop(columns='id').sort_values(by = ['userId','movieId','timestamp']).reset_index(
    drop = True)
tags = tags.sort_values(by = ['userId','movieId','timestamp']).reset_index(drop = True)

In [43]:
(tags_table == tags).sum()

userId       3683
movieId      3683
tag          3683
timestamp    3683
dtype: int64

## Links

In [44]:
links_table = pd.read_sql_table(
    "links",
    con=engine,
    schema = "movie_recommender"
)

In [45]:
links_table = links_table.drop(columns='id').sort_values(by = ['movieId']).reset_index(
    drop = True)
links = links.sort_values(by = ['movieId']).reset_index(drop = True)

In [46]:
(links_table == links).sum()

movieId    9742
imdbId     9742
tmdbId     9734
dtype: int64